This notebook is being used to determine the best approach to calculating AQI.  The methods developed here will be used in the pipeline.

In [43]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3  
from aqipy import aqi_us
from aqipy import aqi_cn
from aqipy import aqi_in
from aqipy import aqi_au
from aqipy import caqi_eu
from aqipy import cai_kr
from aqipy import daqi_uk
from aqipy import aqhi_hk
from aqipy import aqhi_ca
from aqipy import psi_sg

In [44]:
# Load data
import os

# connect to the db and load the data

conn = sqlite3.connect('./data/bkk_aqi.db')
c = conn.cursor()
aqi_data = pd.read_sql_query("SELECT * FROM aqi_data", conn)
aqi_data['time_iso'] = pd.to_datetime(aqi_data['time_iso'])
aqi_data = aqi_data.set_index('time_iso')

print(aqi_data.head())

                           id  aqi   co     h  no2    o3  pressure  pm10  \
time_iso                                                                   
2025-02-13 20:00:00+07:00   1  138  0.1  48.0  2.4  19.4    1010.0    58   
2025-02-13 20:00:00+07:00   2  138  0.1  48.0  2.4  19.4    1010.0    58   
2025-02-13 20:00:00+07:00   3  138  0.1  48.0  2.4  19.4    1010.0    58   
2025-02-13 20:00:00+07:00   4  138  0.1  48.0  2.4  19.4    1010.0    58   
2025-02-13 20:00:00+07:00   5  138  0.1  48.0  2.4  19.4    1010.0    58   

                           pm25  so2  temperature  wind  city_geo_lat  \
time_iso                                                                
2025-02-13 20:00:00+07:00   138  0.6         30.0   2.0     13.756331   
2025-02-13 20:00:00+07:00   138  0.6         30.0   2.0     13.756331   
2025-02-13 20:00:00+07:00   138  0.6         30.0   2.0     13.756331   
2025-02-13 20:00:00+07:00   138  0.6         30.0   2.0     13.756331   
2025-02-13 20:00:00+07:00   1

In [45]:
# Pull the most recent 1 hours of data
recent_data_1h = aqi_data[aqi_data.index >= (aqi_data.index.max() - pd.Timedelta(hours=1))]

#calculate the mean of the most recent 1 hour of data for o3, no2, and so2
mean_o3_1h = recent_data_1h['o3'].mean()
mean_no2_1h = recent_data_1h['no2'].mean()
mean_so2_1h = recent_data_1h['so2'].mean()

# Calculate the max of the most recent 1 hour of data for o3, no2, and so2
max_o3_1h = recent_data_1h['o3'].max()
max_no2_1h = recent_data_1h['no2'].max()
max_so2_1h = recent_data_1h['so2'].max()

# Pull the most recent 3 hours of data
recent_data_3h = aqi_data[aqi_data.index >= (aqi_data.index.max() - pd.Timedelta(hours=3))]

# Calculate the mean of the recent 3 hours of data for pm10, pm25, o3, so2, no2 and co
mean_pm10_3h = recent_data_3h['pm10'].mean()
mean_pm25_3h = recent_data_3h['pm25'].mean()
mean_o3_3h = recent_data_3h['o3'].mean()
mean_so2_3h = recent_data_3h['so2'].mean()
mean_no2_3h = recent_data_3h['no2'].mean()
mean_co_3h = recent_data_3h['co'].mean()

# Pull the most recent 8 hours of data
recent_data_8h = aqi_data[aqi_data.index >= (aqi_data.index.max() - pd.Timedelta(hours=8))]

# Calculate the mean of the recent 8 hours of data for o3 and co
mean_o3_8h = recent_data_8h['o3'].mean()
mean_co_8h = recent_data_8h['co'].mean()

# Pull the most recent 24 hours of data
recent_data_24h = aqi_data[aqi_data.index >= (aqi_data.index.max() - pd.Timedelta(hours=24))]

# Calculate the mean of the recent 24 hours of data for pm10, pm25, and so2
mean_pm10_24h = recent_data_24h['pm10'].mean()
mean_pm25_24h = recent_data_24h['pm25'].mean()
mean_so2_24h = recent_data_24h['so2'].mean()

In [46]:
# Get AQI using the us calculations 
aqi, aqi_data_us = aqi_us.get_aqi(o3_8h=mean_o3_8h, co_8h=mean_co_8h)

print('AQI O3:', aqi_data_us['o3_8h'][0])
print('Effects O3:', aqi_data_us['o3_8h'][1])
print('Cautions O3:', aqi_data_us['o3_8h'][2])
print('AQI CO:', aqi_data_us['co_8h'][0])
print('Effects CO:', aqi_data_us['co_8h'][1])
print('Cautions CO:', aqi_data_us['co_8h'][2])

AQI O3: 300
Effects O3: Increasingly severe symptoms and impaired breathing likely in people with lung disease (such as asthma), children, older adults, people who are active outdoors (including outdoor workers), people with certain genetic variants, and people with diets limited in certain nutrients; increasing likelihood of respiratory effects in general population.
Cautions O3: People with lung disease (such as asthma), children, older adults, people who are active outdoors (including outdoor workers), people with certain genetic variants, and people with diets limited in certain nutrients should avoid all outdoor exertion; everyone else should reduce outdoor exertion.
AQI CO: 1
Effects CO: 
Cautions CO: 
